In [0]:
dbutils.widgets.removeAll()

In [0]:
dbutils.widgets.text("config", "/Workspace/Users/alex.lopes@databricks.com/WorkspaceInventory/conf/")
widgets = dbutils.widgets.getAll()

In [0]:
from databricks.sdk import WorkspaceClient, AccountClient
from pprint import pprint
import yaml
import traceback
from pyspark.sql.functions import *
import os

In [0]:
print("Config:", widgets["config"])
configs = {}
for file in os.listdir(widgets["config"]):
  print("File:", file)
  with open(widgets['config'] + file, "r") as f:
    configs[file] = yaml.safe_load(f.read())

pprint(configs)

Config: /Workspace/Users/alex.lopes@databricks.com/WorkspaceInventory/conf/
File: alerts.yml
File: clusters.yml
File: experiments.yml
File: recipients.yml
File: workflows.yml
File: storage_credentials.yml
File: warehouses.yml
File: secrets.yml
File: shares.yml
File: repos.yml
File: pipelines.yml
File: notebooks.yml
File: models.yml
File: ip_access_lists.yml
File: instance_pools.yml
File: global_init_scripts.yml
{'alerts.yml': [{'api': {'call': 'w.alerts.list',
                         'write': {'mode': 'overwrite',
                                   'table': 'shared.bigc.alerts'}}}],
 'clusters.yml': [{'api': {'call': 'w.clusters.list',
                           'write': {'mode': 'overwrite',
                                     'table': 'shared.bigc.clusters'}}}],
 'experiments.yml': [{'api': {'call': 'w.experiments.list_experiments',
                              'write': {'mode': 'overwrite',
                                        'table': 'shared.bigc.experiments'}}}],
 'global_i

In [0]:

columns = ["object_type"]
objects = []

for key, values in configs.items():
  key = key.replace(".yaml", "").replace(".yml", "")
  for conf in values:
    table = conf['api']['write']['table']
    print("Table:", table)
    if spark.catalog.tableExists(table):      
      objects.append(f"select '{key}' as object_type from {table}")

objects = " union all ".join(objects)

spark.sql("create or replace view shared.bigc.objects as " + objects)
df_objects = spark.read.table("shared.bigc.objects") 

df_objects.display()

Table: shared.bigc.alerts
Table: shared.bigc.clusters
Table: shared.bigc.experiments
Table: shared.bigc.recipients
Table: shared.bigc.workflows
Table: shared.bigc.storage_credentials
Table: shared.bigc.warehouses
Table: shared.bigc.secrets
Table: shared.bigc.shares
Table: shared.bigc.repos
Table: shared.bigc.pipelines
Table: shared.bigc.notebooks
Table: shared.bigc.models
Table: shared.bigc.ip_access_lists
Table: shared.bigc.instance_pools
Table: shared.bigc.global_init_scripts


object_type
alerts
alerts
alerts
alerts
alerts
alerts
alerts
alerts
alerts
alerts


In [0]:
%sql drop table shared.bigc.objects

In [0]:
df_objects.write.mode("overwrite").saveAsTable("shared.bigc.objects")